In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
import pandas as pd
from pathlib import Path
from os import getcwd, remove, chdir
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))
current_dir = getcwd()
print(current_dir)

medir = lambda x: [ y for y in dir(x) if y[0] != '_' ] 

In [ ]:
from bulletin.services.metabase import Metabase 

In [ ]:
mb = Metabase()

In [ ]:
schema = mb.notificacao_information_schema(replace=True)
schema

In [ ]:
schema_original = mb.notificacao_information_schema(replace=False)
schema_original

In [ ]:
schema_original.loc[schema_original['notifica']==1]

In [ ]:
novas_colunas = schema.loc[~schema['column'].isin(schema_original['column']),['column','dtypes','constraint_name','foreign_table_name','foreign_column_name']]
novas_colunas['notifica'] = 1
novas_colunas['casos_confirmados'] = 1
novas_colunas['csv_geral'] = 1
novas_colunas['id'] = [ x for x in range(schema_original.loc[schema_original['notifica']==1,'id'].iloc[-1]+1,schema_original.loc[schema_original['notifica']==1,'id'].iloc[-1]+1+len(novas_colunas))]
novas_colunas['group_name'] = 'novas_colunas'
novas_colunas

In [ ]:
if len(novas_colunas) > 0:
    schema_original.loc[schema_original['notifica']==0,'id'] = [ x for x in range(novas_colunas['id'].iloc[-1]+1,novas_colunas['id'].iloc[-1]+1+len(schema_original.loc[schema_original['notifica']==0]))]
    novo_schema = pd.concat([schema_original.set_index('id'),novas_colunas.set_index('id')]).sort_index().reset_index()
    mb.update_notificacao_schema(novo_schema)
    novo_schema

In [ ]:
mb.download_tables()